In [31]:
import layer
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## Fetch datasets

In [4]:
application = layer.get_dataset("application_train").to_pandas()

In [80]:
application.sample()

,AMT_ANNUITY,AMT_DOWN_PAYMENT,DAYS_FIRST_DRAWING,NAME_CASH_LOAN_PURPOSE,DAYS_TERMINATION,CHANNEL_TYPE,AMT_CREDIT,HOUR_APPR_PROCESS_START,DAYS_DECISION,NAME_TYPE_SUITE,...,DAYS_LAST_DUE_1ST_VERSION,SK_ID_PREV,NFLAG_LAST_APPL_IN_DAY,AMT_GOODS_PRICE,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,NAME_YIELD_GROUP,NAME_PORTFOLIO,FLAG_LAST_APPL_PER_CONTRACT,INDEX
24624,3982.095,0.0,365243.0,XAP,-633.0,Country-wide,18733.5,10,-819,Unaccompanied,...,-638.0,2473114,1,16605.0,NaN,40,high,POS,Y,482885


In [15]:
installment_payments = layer.get_dataset("installments_payments").to_pandas()

In [16]:
installment_payments.sample()

,SK_ID_CURR,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,SK_ID_PREV,INDEX
315812,372409,-511.0,241.605,241.605,78,0.0,-511.0,1109581,10494728


In [8]:
previous_application = layer.get_dataset("previous_application").to_pandas()

In [9]:
previous_application.sample()

,AMT_ANNUITY,AMT_DOWN_PAYMENT,DAYS_FIRST_DRAWING,NAME_CASH_LOAN_PURPOSE,DAYS_TERMINATION,CHANNEL_TYPE,AMT_CREDIT,HOUR_APPR_PROCESS_START,DAYS_DECISION,NAME_TYPE_SUITE,...,DAYS_LAST_DUE_1ST_VERSION,SK_ID_PREV,NFLAG_LAST_APPL_IN_DAY,AMT_GOODS_PRICE,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,NAME_YIELD_GROUP,NAME_PORTFOLIO,FLAG_LAST_APPL_PER_CONTRACT,INDEX
10994,9806.22,0.0,365243.0,XAP,365243.0,Country-wide,86337.45,11,-204,None,...,103.0,1288419,1,86337.45,NaN,29,low_normal,POS,Y,1644868


In [10]:
bureau = layer.get_dataset("bureau").to_pandas()

In [84]:
bureau.sample()

,AMT_ANNUITY,CREDIT_ACTIVE,SK_ID_BUREAU,DAYS_ENDDATE_FACT,SK_ID_CURR,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM_LIMIT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM_DEBT,DAYS_CREDIT_UPDATE,DAYS_CREDIT,CREDIT_TYPE,AMT_CREDIT_SUM,CREDIT_CURRENCY,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_ENDDATE,INDEX
31388,0.0,Closed,5466695,-63.0,418659,0,NaN,NaN,0,0.0,-63,-92,Microloan,31500.0,currency 1,0.0,NaN,899053


## Fetch features

In [12]:
previous_application_features = layer.get_featureset("previous_application_features").to_pandas()

In [13]:
previous_application_features.sample()

,INDEX,APPLIED_AWARDED_AMOUNT_DIFF,GOODS_PRICE_APPLIED_DIFF
10880,1641002,42619.5,0.0


In [18]:
application_features = layer.get_featureset("application_features").to_pandas()

In [19]:
application_features.sample()

,INDEX,ANNUITY_INCOME_RATIO,CREDIT_INCOME_RATIO,CREDIT_TERM,DAYS_EMPLOYED_RATIO,GOODS_PRICE_LOAN_DIFFERENCE
7970,150130,0.259773,2.613227,0.099407,0.146792,-24709.5


In [20]:
bureau_features = layer.get_featureset("bureau_features").to_pandas()

In [21]:
bureau_features.sample()

,INDEX,CREDIT_LIMIT_ABOVE_ZERO,HAS_DEBT,HAS_OVERDUE_DEBT
38944,1178933,0,0,0


##  Fetch model

In [85]:
credit_model = layer.get_model("credit_score_model:79.4")

In [86]:
print(credit_model.parameters)
print(credit_model.metrics)

{'l2_regularization': '0.028679718370026797', 'max_depth': '5', 'test_size': '0.3', 'random_state': '42', 'learning_rate': '0.7232099468700303', 'max_iter': '500', 'min_samples_leaf': '50'}
{'avg_precision': [(1637776167935, 0.8552733368990358)], 'roc_auc_score': [(1637776167946, 0.9728203892885045)]}


In [87]:
trained_classifier = credit_model.get_train()

In [88]:
data = np.array([[1731690, -1916.0,-1953.0,6953.31,6953.31,1731690,0, 0  ,1731690 ,0.2976,7.47512,0.039812,1731690,0.189752,-161451.0,1731690,1731690,1731690,1731690,1,-16074.0, 1731690, 0.0, 1 ]])


In [90]:
categories = []
transformer = ColumnTransformer(
        transformers=[('cat', OneHotEncoder(handle_unknown='ignore', drop="first"), categories)],
        remainder='passthrough')

In [91]:
data = transformer.fit_transform(data)

In [92]:
trained_classifier.predict(data)

array([1])

In [95]:
trained_classifier.predict_proba(data)

array([[0., 1.]])

In [94]:
!curl --header "Content-Type: application/json; format=pandas-records" \
  --request POST \
  --data '[{"OWN_CAR_AGE":10, "APPLIED_AWARDED_AMOUNT_DIFF":20, "REGION_RATING_CLIENT_W_CITY":30, "CREDIT_INCOME_RATIO":0.25,"CREDIT_TERM":300, "DAYS_BIRTH":14600, "cluster":0, "DAYS_EMPLOYED_RATIO":10000, "AMT_INSTALMENT":20, "DAYS_ID_PUBLISH":3000, "GOODS_PRICE_LOAN_DIFFERENCE":52, "AMT_PAYMENT":3000, "HAS_DEBT":0, "REG_CITY_NOT_WORK_CITY":1, "AMT_CREDIT_SUM_OVERDUE":0, "FLAG_DOCUMENT_3":1, "DAYS_ENTRY_PAYMENT":45789, "DAYS_REGISTRATION":9000, "LIVE_CITY_NOT_WORK_CITY":0, "DAYS_INSTALMENT":122, "ANNUITY_INCOME_RATIO":0.247, "GOODS_PRICE_APPLIED_DIFF":-258, "REGION_RATING_CLIENT":1, "CREDIT_LIMIT_ABOVE_ZERO":0}]' \
https://7d674b54-c62f-4333-92eb-7f316445d77b.inferences.beta.layer.co/invocations

[0]